In [ ]:
# change to use your AKV endpoint
AKV_ENDPOINT = "https://xxx.vault.azure.net/"

#cut off date the data ingestion CIJ data, any record before this date will be ignored
cutoffdate = "2024-05-09 09:46:55"

# Define the JDBC URL for your PostgrSQL 
jdbc_url = "jdbc:postgresql://xxx.postgres.database.azure.com:5432/fabricoutput"

In [ ]:
import os
#customer should setup Azure Key Vault and store secret in AKV instead of hardcoding here
#Please refer to this guildeline https://www.datasarva.com/fabric-notebook-azurekeyvault/

os.environ["AZURE_CLIENT_ID"] = mssparkutils.credentials.getSecret(AKV_ENDPOINT, 'AZURE-CLIENT-ID')
os.environ["AZURE_TENANT_ID"] = mssparkutils.credentials.getSecret(AKV_ENDPOINT, 'AZURE-TENANT-ID')
os.environ["AZURE_CLIENT_SECRET"] = mssparkutils.credentials.getSecret(AKV_ENDPOINT, 'AZURE-CLIENT-SECRET')


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from notebookutils import mssparkutils

#get workspace name
workspaceidty = mssparkutils.env.getWorkspaceName()


# Initialize Spark session
spark = SparkSession.builder \
    .appName("PostgreSQL Example") \
    .config("spark.jars", "/path/to/postgresql-42.2.18.jar") \
    .getOrCreate()

# Define the JDBC URL and connection properties
jdbc_url = "jdbc:postgresql://denlaipostgresqlwestus3.postgres.database.azure.com:5432/fabricoutput"

from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

token = credential.get_token("https://ossrdbms-aad.database.windows.net/.default").token

connection_properties = {
    "user": workspaceidty,
    "password": token,
    "driver": "org.postgresql.Driver"
}

#print("token = ", token)

In [ ]:
# just for debug
# if you want to reset the lasttimestamp stored in table last_ts_opened, then unfreeze this cell 
schema = StructType([
    StructField("lasttimestamp", StringType(), True)
])

# Create the DataFrame
data = [(cutoffdate,)]
write_ts_df = spark.createDataFrame(data, schema)

# Write DataFrame to PostgreSQL
write_ts_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.last_ts_opened") \
    .options(**connection_properties) \
    .mode("overwrite") \
    .save()

In [ ]:

# Query to check if the table exists
query = """
SELECT EXISTS (
    SELECT 1
    FROM information_schema.tables 
    WHERE table_schema = 'public'
    AND table_name = 'last_ts_opened'
) AS table_exists
"""

# Execute the query
dfchecktbl = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", query) \
    .options(**connection_properties) \
    .load()

# Check if the table exists
table_exists = dfchecktbl.collect()[0]["table_exists"]

if not table_exists:
    print("table_exists = NO", )
    # Define the schema
    schema = StructType([
        StructField("lasttimestamp", StringType(), True)
    ])

    # Create the DataFrame
    data = [(cutoffdate,)]
    write_ts_df = spark.createDataFrame(data, schema)

    # Write DataFrame to PostgreSQL
    write_ts_df.write \
      .format("jdbc") \
      .option("url", jdbc_url) \
      .option("dbtable", "public.last_ts_opened") \
      .options(**connection_properties) \
      .mode("overwrite") \
      .save()

    last_ts = cutoffdate
    print("last_ts = ", last_ts)
else:
    print("table_exists = YES", )
    # Read the table
    load_ts_df = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "public.last_ts_opened") \
        .options(**connection_properties) \
        .load()

    # Show the first row value
    first_row = load_ts_df.first()
    last_ts = first_row["lasttimestamp"]
    print("last_ts = ", last_ts)
    load_ts_df.show()

In [ ]:
from pyspark.sql.functions import col, max

# Filter the DataFrame to include only records with Timestamp > '2024-05-09 09:46:55'
filtered_df = spark.read.parquet("Files/EmailOpened").filter(col("Timestamp") > last_ts)

# Check if the DataFrame is empty
if filtered_df.count() == 0:
    print("filtered_df is empty. Exiting notebook with positive finish state.")
    mssparkutils.notebook.exit("Success, filtered_df is empty")

# Deduplicate the DataFrame
print("before dedup count = ", filtered_df.count())
dup_filtered_df = filtered_df.dropDuplicates()
print("after dedup count = ", dup_filtered_df.count())

# Get the maximum value of the Timestamp column
if dup_filtered_df.count() > 0:
    newlast_ts = dup_filtered_df.agg(max(col("Timestamp"))).collect()[0][0]
    print("Last Timestamp: ", newlast_ts)
else:
    print("dup_filtered_df DataFrame is empty after deduplication.")
    mssparkutils.notebook.exit("Success, dup_filtered_df is empty")    

if not isinstance(newlast_ts, str):
    newlast_ts = str(newlast_ts)

print("newlast_ts", newlast_ts)

# Show the filtered DataFrame
#dup_filtered_df.show()

In [ ]:
#Sample code for manipulating columns

# Get a list of all column names
#columns = dup_filtered_df.columns
#print("Columns: ", columns)

# Rename columns
#df = dup_filtered_df.withColumnRenamed("old_name1", "new_name1")    

# Select only the "ActivityId" and "Timestamp" columns
#selected_df = dup_filtered_df.select("ActivityId", "Timestamp")

# Show the DataFrame with only the selected columns
#selected_df.show()

In [ ]:
# Just for debug
# Print the count
print("filtered_record_count:", filtered_df.count())
print("dedup_record_count:", dup_filtered_df.count())


In [ ]:
# Write DataFrame to PostgreSQL
dup_filtered_df.write \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("dbtable", "public.emailopened") \
  .options(**connection_properties) \
  .mode("append") \
  .save()


In [ ]:
# Write new timestamp 

schema = StructType([
    StructField("lasttimestamp", StringType(), True)
])
print("newlast_ts", newlast_ts)
data = [(newlast_ts,)]
newts_df = spark.createDataFrame(data, schema)


newts_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.last_ts_opened") \
    .options(**connection_properties) \
    .mode("overwrite") \
    .save()



In [ ]:
# just for debug
# Validate the records in db
# Read the table back from PostgreSQL
read_updated_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.emailopened") \
    .options(**connection_properties) \
    .load()

# Get the count of records in the original DataFrame
dedup_count = dup_filtered_df.count()

# Get the count of records in the table read from PostgreSQL
added_count = read_updated_df.count()

# Validate the counts
if dedup_count == added_count:
    print("Validation successful: The number of records matches.")
else:
    print(f"Validation failed: Dedupfiltered count = {dedup_count}, added_count = {added_count}")

In [ ]:
# just for debug
# Validate the timestamp records in db
read_ts_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.last_ts_opened") \
    .options(**connection_properties) \
    .load()


# Show the first row value
first_row = read_ts_df.first()
newlast_ts = first_row["lasttimestamp"]
print("newlast_ts = ", newlast_ts)


In [ ]:
#If Fabric Capacity is F2/F4 and you have more than one concurrently session
# you need to unremark mssparkutils.session.stop() this line to let Fabric terminate one session before start another one
# You need to wait for new session startup if you stop the session

# release session resources
# mssparkutils.session.stop()